## Overview

* Consume the delta file created in the previous step and modified dataa
* Save data as bronze back to the lake

## Import libraries

In [0]:
from pyspark.sql.functions import col, to_timestamp
from pyspark.sql.types import *
import time

In [0]:
database_name = "MultiTouch"
raw_data_path = "/mnt/da-global-raw-box/POC/MultiTouch"
bronze_tbl_path = "/mnt/da-global-raw-box/POC/MultiTouchBronze"

## Load Data 

  1. Read the synthetic data into a dataframe 
  3. Add two new columns to the dataframe

In [0]:
raw_data_df = spark.read \
            .format("delta") \
            .load(raw_data_path) 

In [0]:
raw_data_df = raw_data_df \
              .withColumn("time", to_timestamp(col("time"),"yyyy-MM-dd HH:mm:ss")) \
              .withColumn("conversion", col("conversion").cast("int"))

## Write Data to Delta Lake
  

In [0]:
raw_data_df \
  .write \
  .format("delta") \
  .save(bronze_tbl_path)

## Create Database

* This will simplify data access in other steps

In [0]:
# Delete the old database and tables if needed
_ = spark.sql('DROP DATABASE IF EXISTS {} CASCADE'.format(database_name))

# Create database to house tables
_ = spark.sql('CREATE DATABASE {}'.format(database_name))

## Create bronze-level table in Delta format

* The nomenclature of bronze, silver, and gold tables correspond with a commonly used data modeling approach known as multi-hop architecture. 
  * Additional information about this pattern can be found [here](https://databricks.com/blog/2019/08/14/productionizing-machine-learning-with-delta-lake.html).

In [0]:
# Create bronze table
_ = spark.sql('''
  CREATE TABLE `{}`.bronze
  USING DELTA 
  LOCATION '{}'
  '''.format(database_name,bronze_tbl_path))

## View the bronze table


In [0]:
%sql

select * from MultiTouch.bronze

uid,time,interaction,channel,conversion
177ce51f7abf4f64a2066af7bde04f56,2020-06-07T22:07:01.000+0000,impression,Affiliates,0
bf59afcf0b7b4a65b53ae99fecf57e92,2020-05-28T08:53:15.000+0000,impression,Affiliates,0
bf59afcf0b7b4a65b53ae99fecf57e92,2020-06-08T04:51:37.000+0000,impression,Email,0
96fc92cc66a84997ab682afde2c667e8,2020-06-01T08:53:39.000+0000,impression,Google Display Network,0
96fc92cc66a84997ab682afde2c667e8,2020-06-05T10:21:48.000+0000,impression,Affiliates,0
bd0b9e0e622b4d03858cf2c69304b5cf,2020-05-23T00:25:59.000+0000,impression,Social Network,0
bd0b9e0e622b4d03858cf2c69304b5cf,2020-05-24T09:40:38.000+0000,impression,Google Display Network,0
bd0b9e0e622b4d03858cf2c69304b5cf,2020-05-24T10:40:54.000+0000,impression,Affiliates,0
bd0b9e0e622b4d03858cf2c69304b5cf,2020-05-30T16:55:50.000+0000,impression,Search Engine Marketing,0
4d6bfa53b9294347a69443f9f4bf5476,2020-05-30T15:36:07.000+0000,impression,Social Network,0
